Dataset: https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset

Variáveis disponíveis:

    ID: ID of each client
    LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
    SEX: Gender (1=male, 2=female)
    EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
    MARRIAGE: Marital status (1=married, 2=single, 3=others)
    AGE: Age in years
    PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)
    PAY_2: Repayment status in August, 2005 (scale same as above)
    PAY_3: Repayment status in July, 2005 (scale same as above)
    PAY_4: Repayment status in June, 2005 (scale same as above)
    PAY_5: Repayment status in May, 2005 (scale same as above)
    PAY_6: Repayment status in April, 2005 (scale same as above)
    BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
    BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
    BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
    BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
    BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
    BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
    PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
    PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
    PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
    PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
    PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
    PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
    default.payment.next.month: Default payment (1=yes, 0=no)

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


import mlflow
from mlflow.models import infer_signature

In [8]:
def total_cost(y_test, y_preds, threshold = 0.5):
    
    tn, fp, fn, tp = confusion_matrix(y_test == 1, y_preds > threshold).ravel()
    
    cost_fn = fn*3000
    cost_fp = fp*1000
    
    return cost_fn + cost_fp

In [9]:
ROOT_PATH = '../data/'
PATH = ROOT_PATH + 'lending_data.csv'
TARGET_COL = 'default.payment.next.month'

SEED = 42

In [10]:
df = pd.read_csv(PATH)

In [11]:
df = df.drop('ID', axis = 1)

In [12]:
from pathlib import Path

# uri = "../../mlruns"

# Path(uri).mkdir(parents=True, exist_ok=True)

# mlflow.set_tracking_uri(uri)

from pathlib import Path

uri = "http://0.0.0.0:5001"

mlflow.set_tracking_uri(uri)

In [13]:
mlflow.set_experiment("Good Clients Prediction Experiment")

2025/04/10 18:50:56 INFO mlflow.tracking.fluent: Experiment with name 'Good Clients Prediction Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/249634156294266374', creation_time=1744307456127, experiment_id='249634156294266374', last_update_time=1744307456127, lifecycle_stage='active', name='Good Clients Prediction Experiment', tags={}>

In [14]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = SEED)

In [15]:
X_train = train_set.drop(['default.payment.next.month'], axis = 'columns')
y_train = train_set['default.payment.next.month']

X_test = test_set.drop(['default.payment.next.month'], axis = 1)
y_test = test_set['default.payment.next.month']

In [16]:
run = mlflow.start_run(run_name="Logistic Forest")
RUN_ID = run.info.run_uuid
RUN_ID

'4bed008226e64b92932d403b4c3e70a8'

In [17]:
scaler = MinMaxScaler()

features_names = X_train.columns

X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train, columns = features_names)

X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test, columns = features_names)

In [18]:
# guardarmos o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source=PATH, targets=TARGET_COL, name="Lending Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=PATH, targets=TARGET_COL, name="Lending Dataset")
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

signature = infer_signature(X_train, y_train)

/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/lending_data.csv'. Exception: 
  return _dataset_source_registry.resolve(
/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/lending_data.csv'. Exception: 
  return _dataset_source_registry.resolve(
/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset

In [22]:
lr = LogisticRegression(max_iter = 500, solver = 'lbfgs', random_state = SEED, class_weight = 'balanced' )

parameters = {'C':[0.001, 0.01, 0.1, 1, 10, 100]}

clf_lr = GridSearchCV(lr, parameters, cv = 5).fit(X_train, y_train)

/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

In [23]:
mlflow.sklearn.log_model(clf_lr.best_estimator_, artifact_path="logistic_reg", registered_model_name="logistic_reg", signature=signature)

params=clf_lr.best_estimator_.get_params()
mlflow.log_params(params)
params

Successfully registered model 'logistic_reg'.
2025/04/10 18:55:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_reg, version 1
Created version '1' of model 'logistic_reg'.


{'C': 0.1,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 500,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [24]:
y_preds = clf_lr.best_estimator_.predict(X_test)

In [25]:
mlflow.log_metric("accuracy", accuracy_score(y_test, y_preds))
mlflow.log_metric("recall", recall_score(y_test, y_preds))
mlflow.log_metric("precision", precision_score(y_test, y_preds))
mlflow.log_metric("f1", f1_score(y_test, y_preds))
mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_preds))
mlflow.log_metric("total_cost", total_cost(y_test, y_preds, threshold = 0.5))

In [26]:
mlflow.end_run()

🏃 View run Logistic Forest at: http://0.0.0.0:5001/#/experiments/249634156294266374/runs/4bed008226e64b92932d403b4c3e70a8
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/249634156294266374


# Pipeline

In [27]:
run = mlflow.start_run(run_name="Logistic Forest - Pipeline")
RUN_ID = run.info.run_uuid
RUN_ID

'57289f14dbad401485996fa090eb6ce4'

In [28]:
# guardarmos o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source=PATH, targets=TARGET_COL, name="Lending Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=PATH, targets=TARGET_COL, name="Lending Dataset")
mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

signature = infer_signature(X_train, y_train)

/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/lending_data.csv'. Exception: 
  return _dataset_source_registry.resolve(
/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/lending_data.csv'. Exception: 
  return _dataset_source_registry.resolve(
/opt/anaconda3/envs/OML_Latest/lib/python3.12/site-packages/mlflow/data/dataset

In [29]:
lr_pipeline = Pipeline(
    steps=[
        ("scaler", MinMaxScaler()),
        ("logistic_regression", clf_lr.best_estimator_)
])
lr_pipeline.fit(X_train, y_train)
mlflow.sklearn.log_model(lr_pipeline, artifact_path="lr_pipeline", registered_model_name="logistic_reg")
lr_pipeline

2025/04/10 18:58:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'logistic_reg' already exists. Creating a new version of this model...
2025/04/10 18:58:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_reg, version 2
Created version '2' of model 'logistic_reg'.


Pipeline(steps=[('scaler', MinMaxScaler()),
                ('logistic_regression',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    max_iter=500, random_state=42))])

In [30]:
params=lr_pipeline.get_params()

modified_params = {}
for k, v in params.items():
    new_key = k.replace("logistic_regression__", '')
    modified_params[new_key] = v

mlflow.log_params(modified_params)
modified_params

{'memory': None,
 'steps': [('scaler', MinMaxScaler()),
  ('logistic_regression',
   LogisticRegression(C=0.1, class_weight='balanced', max_iter=500,
                      random_state=42))],
 'transform_input': None,
 'verbose': 0,
 'scaler': MinMaxScaler(),
 'logistic_regression': LogisticRegression(C=0.1, class_weight='balanced', max_iter=500,
                    random_state=42),
 'scaler__clip': False,
 'scaler__copy': True,
 'scaler__feature_range': (0, 1),
 'C': 0.1,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 500,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'warm_start': False}

In [31]:
y_preds = lr_pipeline.predict(X_test)

In [32]:
mlflow.log_metric("accuracy", accuracy_score(y_test, y_preds))
mlflow.log_metric("recall", recall_score(y_test, y_preds))
mlflow.log_metric("precision", precision_score(y_test, y_preds))
mlflow.log_metric("f1", f1_score(y_test, y_preds))
mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_preds))
mlflow.log_metric("total_cost", total_cost(y_test, y_preds, threshold = 0.5))

In [33]:
mlflow.end_run()

🏃 View run Logistic Forest - Pipeline at: http://0.0.0.0:5001/#/experiments/249634156294266374/runs/57289f14dbad401485996fa090eb6ce4
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/249634156294266374
